In [ ]:
from pandana.loaders import osm
import folium, requests, json
import pandas as pd
from config import census_key, gkey
from census import Census

# Building and cleaning neighborhood dataframe

In [ ]:
df = pd.read_csv('CSVs/Neighborhoods__Regions_ (1).csv')

In [ ]:
df

In [ ]:
url = 'https://opendata.arcgis.com/datasets/9f50a605cf4945259b983fa35c993fe9_125.geojson'
geo = requests.get(url).json()
print(json.dumps(geo, indent=4))

In [ ]:
names = []
lats = []
lngs = []
for i in geo['features']:
    names.append(i['properties']['NAME'])
    lats.append(i['geometry']['coordinates'][0][0][1])
    lngs.append(i['geometry']['coordinates'][0][0][0])
query_df = pd.DataFrame()
query_df['name'] = names
query_df['lat'] = lats
query_df['lng'] = lngs

pd.set_option('display.max_rows',None)
query_df = query_df.loc[query_df['name'] != 'MC UNCLAIMED #13', :]
query_df = query_df.loc[query_df['name'] != 'CRESTWOOD', :]


In [ ]:
maps_base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
zipcodes = []
for index, row in query_df.iterrows():
    lat = row.lat
    lng = row.lng
    name = row.name
    
    response = requests.get(f"{maps_base_url}latlng={lat},{lng}&key={gkey}").json()
    
    if response['results'][0]['address_components'][-1]['types'][0] == 'postal_code':
        zipcodes.append(response['results'][0]['address_components'][-1]['long_name'])
    elif response['results'][0]['address_components'][-2]['types'][0] == 'postal_code':
        zipcodes.append(response['results'][0]['address_components'][-2]['long_name'])
    else:
        zipcodes.append(response['results'][0]['address_components'][-3]['long_name'])



In [ ]:
query_df['Zipcode'] = zipcodes
query_df = query_df.loc[query_df['Zipcode'] != 'Multnomah County', :]
query_df['name'] = [i.title() for i in query_df['name']]

In [ ]:
query_df.iloc[4,0] = 'Cully'
query_df.iloc[9,0] = 'Sumner'
query_df.iloc[15,0] = 'Parkrose'
query_df.iloc[24,0] = 'Old Town Chinatown'
query_df.iloc[26,0] = 'Buckman'
query_df.iloc[33,0] = 'Brooklyn'
query_df.iloc[38,0] = 'Sellwood-Moreland'
query_df.iloc[56,0] = 'Southwest Hills'
query_df.iloc[64,0] = 'Centennial'
query_df.iloc[69,0] = 'Wilkes'
query_df.iloc[76,0] = 'Irvington'
query_df.iloc[77,0] = 'Sabin'
query_df.iloc[82,0] = 'Lloyd District'
query_df.iloc[83,0] = 'Goose Hollow'
query_df.iloc[86,0] = 'Hosford-Abernathy'
query_df.iloc[90,0] = 'Northwest District'

query_df

# Building and cleaning walk score dataframe

In [ ]:
walkscore_df = pd.read_csv('CSVs/PDX_Neighborhood_Walkability.csv')

In [ ]:
walkscore_df

In [ ]:
merged_df = query_df.merge(walkscore_df, left_on='name', right_on='Name')

In [ ]:
merged_df.drop(columns=['Rank', 'Name', 'Transit Score', 'Bike Score', 'Population'], inplace=True)

In [ ]:
merged_df

# 2012 Census data call

In [ ]:
c = Census(census_key, year=2012)
median_incomes = []
pops = []
white_pop = []
nonwhite_pop = []

for i in merged_df['Zipcode']:

    census_data = c.acs5.get(("NAME", 'B01003_001E','B02001_002E', 'B06011_001E',
                             ), {'for': f'zip code tabulation area:{i}'})
    pops.append(census_data[0]['B01003_001E'])
    white_pop.append(census_data[0]['B02001_002E'])
    nonwhite_pop.append(census_data[0]['B01003_001E'] - census_data[0]['B02001_002E'])
    median_incomes.append(census_data[0]['B06011_001E'])
census_data

In [ ]:
merged_df['Median Income 2012'] = median_incomes
merged_df['Population 2012'] = pops
merged_df['Non-White Population 2012'] = nonwhite_pop
merged_df['% Non-White 2012'] = (merged_df['Non-White Population 2012']/merged_df['Population 2012'] * 100)

In [ ]:
merged_df

# 2018 Census data call

In [ ]:
c = Census(census_key, year=2018)
median_incomes = []
pops = []
white_pop = []
nonwhite_pop = []

for i in merged_df['Zipcode']:

    census_data = c.acs5.get(("NAME", 'B01003_001E','B02001_002E', 'B06011_001E',
                             ), {'for': f'zip code tabulation area:{i}'})
    pops.append(census_data[0]['B01003_001E'])
    white_pop.append(census_data[0]['B02001_002E'])
    nonwhite_pop.append(census_data[0]['B01003_001E'] - census_data[0]['B02001_002E'])
    median_incomes.append(census_data[0]['B06011_001E'])
census_data

In [ ]:
merged_df['Median Income 2018'] = median_incomes
merged_df['Population 2018'] = pops
merged_df['Non-White Population 2018'] = nonwhite_pop
merged_df['% Non-White 2018'] = (merged_df['Non-White Population 2018']/merged_df['Population 2018'] * 100)

In [ ]:
zips_group = merged_df.groupby('Zipcode')
zipcode_df = pd.DataFrame(zips_group.mean())
zipcode_df

In [ ]:
zipcode_df.to_csv('CSVs/census_walkscore_by_zip.csv')